In [ ]:
import sys
import os
sys.path.append('../..')
#sys.path.append(os.path.join(os.path.dirname(os.path.dirname(__file__))))

import shutil
import json
# from lib.rigidpose.sixd_toolkit import pysixd
from lib.rigidpose.sixd_toolkit.pysixd import inout
from model_conversion_occl2hint import model_conversion_meta
import numpy as np
from PIL import Image
# from matplotlib import pyplot as plt

In [ ]:
with open(os.path.join('/home/lucas/object-pose-estimation/local_conf.json')) as f:
    local_conf = json.load(f)

with open('/home/lucas/object-pose-estimation/meta.json') as f:
    meta = json.load(f)

In [ ]:
def parse_pose(pose_path, parse_extent=False):
    """
    Parses text file for annotated pose, which is returned.
    """
    def read_array(lines):
        return np.array([list(map(float, line.split())) for line in lines])
    if not os.path.exists(pose_path):
        return None
    with open(pose_path) as f:
        lines = f.readlines()

    D = np.diag([1,-1,-1]);

    ret = {}

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        i += 1
        if line == 'rotation:':
            # Transform camera coordinate system for complying conventions
            ret['R'] = np.dot(D, read_array(lines[i:i+3]))
            i += 3
            continue
        elif line == 'center:':
            # Transform camera coordinate system for complying conventions
            ret['t'] = np.dot(D, read_array([lines[i]]).T)
            if not parse_extent:
                break
            i += 1
            continue
        elif line == 'extent:':
            ret['extent'] = read_array([lines[i]]).T
            break
    else:
        # File contains no pose annotation
        return None
    return ret

In [ ]:
def get_pose_paths(seq, frame_idx, pose_annos_path, considered_objs=None):
    """
    Returns a dict of paths for the objects annotated in the corresponding sequence / frame.
    """
    if considered_objs is None:
        considered_objs = sorted(meta['objects'].keys())
    pose_paths = {}
    for obj in considered_objs:
        if meta['objects'][obj]['mesh_id'] is None:
            continue
        pose_path = os.path.join(pose_annos_path, seq, '{:03}'.format(meta['objects'][obj]['mesh_id']), 'info', 'info_{:05}.txt'.format(frame_idx))
        if not os.path.exists(pose_path):
            continue
        pose = parse_pose(pose_path)
        if pose is not None:
            pose_paths[obj] = pose_path
    return pose_paths

In [ ]:
def seg2pixelcount(seg, label):
    silhouette_pixels = np.argwhere(seg == label)
    return silhouette_pixels.shape[0]

In [ ]:
def seg2bb(seg, label):
    silhouette_pixels = np.argwhere(seg == label)
    if silhouette_pixels.shape[0] == 0:
        # Some instances seem to have pose annotation despite not being visible... 100% occluded by other objects?
        return None
    xmin, ymin = np.min(silhouette_pixels, axis=0)
    xmax, ymax = np.max(silhouette_pixels, axis=0)
#     TODO: Should we add some margin..?
    return [xmin, ymin, xmax-xmin, ymax-ymin]

In [ ]:
def bb_from_proj(K, R, t, U, img_width, img_height):
    u = K @ (R @ U + t)
    u /= u[np.newaxis,2,:]

    xmin = int(np.min(u[0,:]))
    xmax = int(np.max(u[0,:]))
    ymin = int(np.min(u[1,:]))
    ymax = int(np.max(u[1,:]))

    xmin, xmax = np.clip([xmin, xmax], 0, img_width-1)
    ymin, ymax = np.clip([ymin, ymax], 0, img_height-1)

#     TODO: Should we add some margin..?
    return [xmin, ymin, xmax-xmin, ymax-ymin]

In [ ]:
NEW_DATA_PATH = '/home/lucas/datasets/pose-data/sixd/occluded-linemod-augmented4'
# OLD_DATA_PATH = '/home/lucas/datasets/pose-data'
# OLD_MODELS_PATH = '/home/lucas/datasets/pose-data/ply-models-ascii'
OLD_SIXD_LINEMOD_PATH = '/home/lucas/datasets/pose-data/sixd/bop-unzipped/hinterstoisser'
# VTX_FRACTION_FOR_PROJ_BB = 0.25

### What objects to consider?

In [ ]:
# Consider only objects for which occluded images exist already (without augmentation)
# considered_objs = sorted(set(meta['sequences']['ape']['objects_annotated_in_sequence']) - {'benchviseblue'})

# Consider all objects present in test sequence
considered_objs = sorted(set(meta['sequences']['benchviseblue']['objects_annotated_in_sequence']) - {'benchviseblue'})

# Exclude benchviseblue (not occluded in test sequence)
considered_objs = sorted(set(considered_objs) - {'benchviseblue'})

already_considered_objs = considered_objs

all_objs = set(meta['objects'])
# Consider all objects
considered_objs = sorted(all_objs)

# Consider all objects except the ones already considered
#considered_objs = sorted(all_objs - set(already_considered_objs))

### Camera info

In [ ]:
cam_yaml = {}
cam_yaml['depth_scale'] = 1.0 # mm -> mm
cam_yaml['width'] = 640
cam_yaml['height'] = 480
cam_yaml['fx'] = meta['camera_calibration']['f_x']
cam_yaml['fy'] = meta['camera_calibration']['f_y']
cam_yaml['cx'] = meta['camera_calibration']['p_x']
cam_yaml['cy'] = meta['camera_calibration']['p_y']

In [ ]:
# "Standard" calibration be used for synthetic rendering
# Calibration should also be store for each frame.
os.makedirs(NEW_DATA_PATH, exist_ok=True)
inout.save_yaml(os.path.join(NEW_DATA_PATH, 'camera.yml'), cam_yaml)

### Read models to memory, convert m -> mm, and store new PLY files. Find model info and store as YAML.

In [ ]:
occl_instead_of_sixd_hinter = False

if occl_instead_of_sixd_hinter:
    depth_rescale_to_mm = 1e3

    models = {}
    model_yaml = {}
    os.makedirs(os.path.join(NEW_DATA_PATH, 'models'), exist_ok=True)
    for obj in considered_objs:
        bounds = meta['objects'][obj]['bounds']

        # SIXD Hinterstoisser
        obj_id = model_conversion_meta[obj]['idx_sixd']
        models[obj] = inout.load_ply(os.path.join(OLD_SIXD_LINEMOD_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)))

        inout.save_ply(
            os.path.join(NEW_DATA_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)),
            models[obj]['pts'],
            pts_colors = models[obj]['colors'],
            pts_normals = models[obj]['normals'],
            faces = models[obj]['faces'],
        )
    #     shutil.copyfile(
    #         os.path.join(OLD_MODELS_PATH, '{:03}.ply'.format(obj_id)),
    #         os.path.join(NEW_DATA_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)),
    #     )
        model_yaml[obj_id] = {
    #         TODO: maximal distance between vertices?
    #         distance_matrix = squareform(pdist(model['pts'], metric='euclidean'))
    #         'diameter': None,
    #         m -> mm
            'min_x': depth_rescale_to_mm*(bounds['x'][0]),
            'min_y': depth_rescale_to_mm*(bounds['y'][0]),
            'min_z': depth_rescale_to_mm*(bounds['z'][0]),
    #         Keypoints to be produced later by dedicated script:
    #         'kp_x': ,
    #         'kp_y': ,
    #         'kp_z': ,
            'size_x': depth_rescale_to_mm*(bounds['x'][1] - bounds['x'][0]),
            'size_y': depth_rescale_to_mm*(bounds['y'][1] - bounds['y'][0]),
            'size_z': depth_rescale_to_mm*(bounds['z'][1] - bounds['z'][0]),
        }
    inout.save_yaml(os.path.join(NEW_DATA_PATH, 'models', 'models_info.yml'), model_yaml)
else:
    old_model_yaml = inout.load_yaml(os.path.join(OLD_SIXD_LINEMOD_PATH, 'models', 'models_info.yml'))
    models = {}
    model_yaml = {}
    os.makedirs(os.path.join(NEW_DATA_PATH, 'models'), exist_ok=True)
    for obj in considered_objs:
        obj_id = model_conversion_meta[obj]['idx_sixd']
        shutil.copyfile(
            os.path.join(OLD_SIXD_LINEMOD_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)),
            os.path.join(NEW_DATA_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)),
        )
        models[obj_id] = inout.load_ply(os.path.join(OLD_SIXD_LINEMOD_PATH, 'models', 'obj_{:02}.ply'.format(obj_id)))
        model_yaml[obj_id] = old_model_yaml[obj_id]

    inout.save_yaml(os.path.join(NEW_DATA_PATH, 'models', 'models_info.yml'), model_yaml)

In [ ]:
"""
Process one frame, move data to new paths, and return YAML annotation contents
"""
def process_frame(
        old_paths,
        new_paths,
        seq_path,
        frame_idx,
    ):

    shutil.copyfile(old_paths['img_path'], new_paths['img_path'])
    shutil.copyfile(old_paths['depth_path'], new_paths['depth_path'])
    shutil.copyfile(old_paths['corrmap_path'], new_paths['corrmap_path'])

    objclass_seg = np.array(Image.open(old_paths['seg_path']))

#     Initialize with zeros (background). To be filled when iterating over instances.
    instance_seg = np.zeros((cam_yaml['height'], cam_yaml['width']), dtype=np.uint8)

    info_yaml_curr = {
        'cam_K': np.array([
            [cam_yaml['fx'],             0.0,    cam_yaml['cx']],
            [0.0,             cam_yaml['fy'],    cam_yaml['cy']],
            [0.0,                        0.0,               1.0],
        ]),
        'depth_scale': cam_yaml['depth_scale'],
    }
    gt_yaml_curr = []
    instance_id = 1
    for obj, pose_path in old_paths['pose_paths'].items():
        obj_id_occl = meta['objects'][obj]['mesh_id']
        assert obj_id_occl == model_conversion_meta[obj]['idx_occl']
        obj_id_sixd = model_conversion_meta[obj]['idx_sixd']

        model = models[obj] if occl_instead_of_sixd_hinter else models[obj_id_sixd]

#         Add current object to instance segmentation map
        instance_seg[objclass_seg == obj_id_occl] = instance_id

        pose = parse_pose(pose_path)
        pose['t'] *= 1e3 # m -> mm

#         obj_bb = seg2bb(instance_seg, instance_id)
#         if obj_bb is None:
#             raise Exception('Object {} missing in seq {}, frame {}'.format(obj, seq, frame_idx))
        obj_bb = bb_from_proj(info_yaml_curr['cam_K'], pose['R'], pose['t'], model['pts'].T, cam_yaml['width'], cam_yaml['height'])

        gt_yaml_curr.append({
            'cam_R_m2c': pose['R'] @ model_conversion_meta[obj]['R_occl2sixd'].T,
            'cam_t_m2c': pose['t'],
            'obj_bb': obj_bb,
            'obj_id': obj_id_sixd,
            'px_count_visib': seg2pixelcount(instance_seg, instance_id),
        })
        instance_id += 1

#     Save instance seg to file
    instance_seg = Image.fromarray(instance_seg)
    instance_seg.save(new_paths['seg_path'])

    return info_yaml_curr, gt_yaml_curr

In [ ]:
def get_old_paths_aug(seq, frame_idx, considered_objs=None):
    return {
        'img_path': os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'rgb', 'augmented_{}.jpg'.format(frame_idx)),
        'seg_path': os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'seg', 'augmented_{}.png'.format(frame_idx)),
        'depth_path': os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'kinect-depth', 'augmented_{:05}.png'.format(frame_idx)),
        'corrmap_path': os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'coords', 'augmented_{:05}.png'.format(frame_idx)),
        'pose_paths': get_pose_paths(seq, frame_idx, os.path.join(local_conf['object_centric_data_aug_path'], 'infos-augmented'), considered_objs=considered_objs),
    }

In [ ]:
def get_old_paths_lm_occl(seq, frame_idx, considered_objs=None):
    return {
        'img_path': os.path.join(local_conf['linemod_path'], 'data', seq, 'data', 'color{}.jpg'.format(frame_idx)),
        'seg_path': os.path.join(local_conf['rendered_path'], seq, 'seg', 'seg_{:05}.png'.format(frame_idx)),
        'depth_path': os.path.join(local_conf['linemod_depth_path'], seq, 'depth', 'depth_{:05}.png'.format(frame_idx)),
        'corrmap_path': os.path.join(local_conf['rendered_path'], seq, 'coords', 'coords_{:05}.png'.format(frame_idx)),
        'pose_paths': get_pose_paths(seq, frame_idx, local_conf['pose_annos_path'], considered_objs=considered_objs),
    }

In [ ]:
def get_old_paths_synthetic(seq, frame_idx, considered_objs=None):
    return {
        'img_path': os.path.join(local_conf['synthetic_path'], 'rendered', seq, 'rgb_with_bg', 'rgb_{:05}.jpg'.format(frame_idx)),
        'seg_path': os.path.join(local_conf['synthetic_path'], 'rendered', seq, 'seg', 'seg_{:05}.png'.format(frame_idx)),
        'depth_path': os.path.join(local_conf['synthetic_path'], 'rendered', seq, 'depth', 'depth_{:05}.png'.format(frame_idx)),
        'corrmap_path': os.path.join(local_conf['synthetic_path'], 'rendered', seq, 'coords', 'coords_{:05}.png'.format(frame_idx)),
        'pose_paths': get_pose_paths(seq, frame_idx, os.path.join(local_conf['synthetic_path'], 'poses'), considered_objs=considered_objs),
    }

In [ ]:
def get_new_paths(seq_path, frame_idx):
    return {
        'img_path': os.path.join(seq_path, 'rgb', '{:06}.png'.format(frame_idx)),
        'seg_path': os.path.join(seq_path, 'instance_seg', '{:06}.png'.format(frame_idx)),
        'depth_path': os.path.join(seq_path, 'depth', '{:06}.png'.format(frame_idx)),
        'corrmap_path': os.path.join(seq_path, 'obj', '{:06}.png'.format(frame_idx)),
    }

In [ ]:
def make_dirs(seq_path):
    os.makedirs(seq_path, exist_ok=True)
    os.makedirs(os.path.join(seq_path, 'rgb'), exist_ok=True)
    os.makedirs(os.path.join(seq_path, 'instance_seg'), exist_ok=True)
    os.makedirs(os.path.join(seq_path, 'depth'), exist_ok=True)
    os.makedirs(os.path.join(seq_path, 'obj'), exist_ok=True)

### Training set - augmented sequences

In [ ]:
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # TESTING
# # Set of sequences coincides with set of objects
# seqs = considered_objs
# for seq in seqs:
# #     Discard sequence if "occluded objects" not present
#     if not len(set(meta['sequences'][seq]['objects_annotated_in_sequence']) & set(seqs)) > 0:
#         continue

#     seq_path = os.path.join(NEW_DATA_PATH, 'train_aug', seq)
#     make_dirs(seq_path)

#     info_yaml = {}
#     gt_yaml = {}

#     indices = [0]
# #     indices = range(len(os.listdir(os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'rgb'))))

#     print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

#     for frame_idx in indices:

#         info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
#             get_old_paths_aug(seq, frame_idx, considered_objs),
#             get_new_paths(seq_path, frame_idx),
#             seq_path,
#             frame_idx,
#         )

# #     inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
# #     inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

In [ ]:
# Set of sequences coincides with set of objects
seqs = considered_objs
for seq in seqs:
#     Discard sequence if "occluded objects" not present
    if not len(set(meta['sequences'][seq]['objects_annotated_in_sequence']) & set(seqs)) > 0:
        continue

    seq_path = os.path.join(NEW_DATA_PATH, 'train_aug', seq)
    make_dirs(seq_path)

    info_yaml = {}
    gt_yaml = {}

    indices = range(len(os.listdir(os.path.join(local_conf['object_centric_data_aug_path'], 'augmented-images', seq, 'rgb'))))

    print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

    for frame_idx in indices:

        info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
            get_old_paths_aug(seq, frame_idx, considered_objs),
            get_new_paths(seq_path, frame_idx),
            seq_path,
            frame_idx,
        )

    inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
    inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

### Training set - unoccluded LINEMOD

In [ ]:
for obj in considered_objs:
    seq = obj
    seq_path = os.path.join(NEW_DATA_PATH, 'train_unoccl', seq)
    make_dirs(seq_path)

    info_yaml = {}
    gt_yaml = {}

    indices = range(meta['sequences'][seq]['sequence_length'])

    print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

    for frame_idx in indices:
        info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
            get_old_paths_lm_occl(seq, frame_idx, [obj]),
            get_new_paths(seq_path, frame_idx),
            seq_path,
            frame_idx,
        )

    inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
    inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

### Training set - occluded ape sequence

In [ ]:
for seq in ['ape']:
    seq_path = os.path.join(NEW_DATA_PATH, 'train_occl', seq)
    make_dirs(seq_path)

    info_yaml = {}
    gt_yaml = {}

    indices = range(meta['sequences'][seq]['sequence_length'])

    print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

    for frame_idx in indices:
        info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
            get_old_paths_lm_occl(seq, frame_idx, considered_objs),
            get_new_paths(seq_path, frame_idx),
            seq_path,
            frame_idx,
        )

    inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
    inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

### Training set - synthetic images

In [ ]:
for seq in ['seq']:
    seq_path = os.path.join(NEW_DATA_PATH, 'train_synthetic', seq)
    make_dirs(seq_path)

    info_yaml = {}
    gt_yaml = {}

    indices = range(40000)

    print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

    for frame_idx in indices:
        info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
            get_old_paths_synthetic(seq, frame_idx, considered_objs),
            get_new_paths(seq_path, frame_idx),
            seq_path,
            frame_idx,
        )

    inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
    inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

### Test set - occluded benchviseblue

In [ ]:
for seq in ['benchviseblue']:
    seq_path = os.path.join(NEW_DATA_PATH, 'test_occl', seq)
    make_dirs(seq_path)

    info_yaml = {}
    gt_yaml = {}

    indices = indices = range(meta['sequences'][seq]['sequence_length'])

    print("Seq: {}, Frames to process: {}.".format(seq, len(indices)))

    for frame_idx in indices:
        info_yaml[frame_idx], gt_yaml[frame_idx] = process_frame(
            get_old_paths_lm_occl(seq, frame_idx, considered_objs),
            get_new_paths(seq_path, frame_idx),
            seq_path,
            frame_idx,
        )

    inout.save_info(os.path.join(seq_path, 'info.yml'), info_yaml)
    inout.save_gt(os.path.join(seq_path, 'gt.yml'), gt_yaml)

In [ ]:
meta